## 1. Connect to a GPU Instance

---



In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-db75aa7c-5480-4d9a-f0b8-a5fa8fd5fdb8)


## 2. Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. Install pix2pixHD repository OR change directory into repo

**Executing this cell will either install the pix2pixHD repo into your Google Drive, or move you into the pix2pixHD repo if it already exists. This cell also installs a python dependency called _dominate_.**


In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-pix2pixHD"):
    %cd "/content/drive/MyDrive/colab-pix2pixHD/pix2pixHD"
    !pip install dominate
    !pip install -r util/requirements.txt
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-pix2pixHD
    %cd colab-pix2pixHD
    !git clone https://github.com/dougrosman/pix2pixHD
    %cd pix2pixHD
    !mkdir generated_videos
    !pip install dominate
    #install python requirements for Derrick Schultz' dataset-tools.py
    #more info: https://github.com/dvschultz/dataset-tools
    !pip install -r util/requirements.txt
else:
    !git clone https://github.com/dougrosman/pix2pixHD
    %cd pix2pixHD
    !mkdir generated_videos
    !pip install dominate
    !pip install -r util/requirements.txt


/content/drive/MyDrive/colab-pix2pixHD/pix2pixHD
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 167 kB 69.7 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 40 kB 5.7 MB/s 
  Created wheel for imutils: filename=imutils-0.5.2-py3-none-any.whl size=24419 sha256=37331208eec54884826881f52f4fb5d10c2cd883d722e5e493449a2182518164
  Stored in directory: /root/.cache/pip/wheels/82/1a/73/4e2703e5864dc22f47a2e6e1da87f750aea2c2cdacc2ece47e
  Created wheel for mac-tag: filename=mac_tag-2020.12.3-py3-none-any.whl size=1928 sha256=cb7703e7999aeae467fe4f913dfc44c32bc1741a6dc9e543c851b97dfb497a2f
  Stored in directory: /root/.cache/pip/wheels/00/4a/7a/29df4b1ab8e94937b4899be4964c7b64ac90c15acdd39f19ff
  Crea

### 4a. Create necessary dataset folders

#### **Set your Dataset Name**

The dataset_name variable you set here will be used throughout the rest of the notebook. Any time you work with a new dataset, change the name here. **_Any instance of $dataset_name you encounter will refer to whatever you set in this code cell._**

In [ ]:
dataset_name = 'textures_dataset' # change 'dataset_name' to whatever you want to call your dataset

#### **Create your dataset folders**
Inside your datasets folder, you'll need a folder for each dataset you work with. The following command creates these folders:
  1. **_train_A_**, for your input images. Place your input images inside that folder.
  2. **_train_B_**, for your output images. Place your output images inside that folder.
  3. **_test_A_**, for you test input images. We won't use this folder until we get the generate steps after training.


In [ ]:
# Run this cell after setting your dataset_name above

!mkdir ./datasets/$dataset_name
!mkdir ./datasets/$dataset_name/train_A
!mkdir ./datasets/$dataset_name/train_B
!mkdir ./datasets/$dataset_name/test_A

mkdir: cannot create directory ‘./datasets/textures_dataset’: File exists
mkdir: cannot create directory ‘./datasets/textures_dataset/train_B’: File exists
mkdir: cannot create directory ‘./datasets/textures_dataset/test_A’: File exists


## 5. Training

### **Some notes on training:** 

### 5a. Set training variables 

In [ ]:
##### REQUIRED: edit these each time you train a new model from scratch!
name = 'diffuse2normal'    # can be whatever you want; name this based on your dataset (e.g. edges2cats)
loadSize = 1024     # The desired width of your outputs (note: images will be cropped to this) Default=1024 
fineSize = 1024      # The desired height of your outputs
                    ## I haven't trained a model with images larger than 1280x720, though it may be possible!
which_epoch = 'latest' # The epoch you wish to resume training from. Keep this set to 'latest' if you want to
                        ## pick up from where you left off. Otherwise, put the number of the .pth file you want
                        ## to resume from (e.g. 10, 20, 30, etc.)

##### OPTIONAL: change these if needed
#resize_or_crop = 'scale_width'   # keeping this unchanged will automatically resize
                                  ## your images to the loadSize and fineSize, then crop to
                                  ## those dimensions. Set to 'none' if your images are
                                  ## already the correct dimensions

resize_or_crop = 'none'
                            

display_freq = 200    # frequency of showing training results on screen
print_freq = 100      # frequency of showing training results on console
save_latest_freq = 1000     # frequency of saving the latest results
                              ##(lower = more frequent saving, 1000 ~ saves every 10 minutes)
save_epoch_freq = 10    # frequency of saving checkpoints at the end of epochs
                        # (1 epoch is completed after going through every image in your data set 1 time)

### 5b. Running the training command

#### **Option 1: Training a new model from scratch**

Unlike StyleGAN, where use transfer learning by resuming training from a pretrained model like FFHQ, with pix2pixHD, it's much more common to train a model from scratch.

In [ ]:
# ONLY RUN THIS IF YOU ARE STARTING A NEW TRAINING
!python train.py --name=$name --dataroot=./datasets/$dataset_name --checkpoints_dir checkpoints --no_instance --label_nc 0 --resize_or_crop=$resize_or_crop --loadSize=$loadSize --fineSize=$fineSize

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: checkpoints
continue_train: False
data_type: 32
dataroot: ./datasets/textures_dataset
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 1024
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: diffuse2normal
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: none
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log

#### **Option 2: Resuming a training**

Run this command if you are resuming training. You should still set your variables above before running this command.

In [ ]:
# ONLY RUN THIS IF YOU ARE RESUMING A TRAINING
!python train.py --name=$name --dataroot=./datasets/$dataset_name --checkpoints_dir checkpoints --no_instance --label_nc 0 --resize_or_crop=$resize_or_crop --continue_train --which_epoch=$which_epoch --loadSize=$loadSize --fineSize=$fineSize

------------ Options -------------
batchSize: 1
beta1: 0.5
checkpoints_dir: checkpoints
continue_train: True
data_type: 32
dataroot: ./datasets/textures_dataset
debug: False
display_freq: 100
display_winsize: 512
feat_num: 3
fineSize: 1024
fp16: False
gpu_ids: [0]
input_nc: 3
instance_feat: False
isTrain: True
label_feat: False
label_nc: 0
lambda_feat: 10.0
loadSize: 1024
load_features: False
load_pretrain: 
local_rank: 0
lr: 0.0002
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: diffuse2normal
ndf: 64
nef: 16
netG: global
ngf: 64
niter: 100
niter_decay: 100
niter_fix_global: 0
no_flip: False
no_ganFeat_loss: False
no_html: False
no_instance: True
no_lsgan: False
no_vgg_loss: False
norm: instance
num_D: 2
output_nc: 3
phase: train
pool_size: 0
print_freq: 100
resize_or_crop: none
save_epoch_freq: 10
save_latest_freq: 1000
serial_batches: False
tf_log:

## 6. Generating Images

### 6a. Preparing your test images

#### **Set your experiment name**

The experiment_name variable you set here will be used throughout the rest of the notebook. Any time you work with a new experiment, change the name here. **_Any instance of $experiment_name you encounter will refer to whatever you set in this code cell._**

In [ ]:
experiment_name = 'stablediffuse2normal_exp' # change 'my_experiment_name' to whatever you want to call your experiment

#### **Set your Dataset Name**

(Note: if you're doing this step in the same session as your training, your dataset_name will have already been set. Feel free to set it again anytime you're generating images using a different dataset/trained model).

The dataset_name variable you set here will be used throughout the rest of the notebook. Any time you work with a new dataset, change the name here. **_Any instance of $dataset_name you encounter will refer to whatever you set in this code cell._**

In [ ]:
dataset_name = 'stable_textures_dataset' # change 'dataset_name' to whatever you want to call your dataset

#### **Option 1:** Generating images from your original training data (the images from your train_A folder)

1. This cell removes any images from your test_A folder (if there are any), and... 
2. Copies all the images from your train_A folder to your test_A folder.
Testing your trained model with the original data set can be useful to see how accurately the model can recreate the training data.

In [ ]:
# remove any images currently in test_A
!rm -v ./datasets/$dataset_name/test_A/*.jpg

# copy images from train_A to test_A
!cp -v ./datasets/$dataset_name/train_A/*.jpg ./datasets/$dataset_name/test_A

rm: cannot remove './datasets/textures_dataset/test_A/*.jpg': No such file or directory
'./datasets/textures_dataset/train_A/Texture (1000).jpg' -> './datasets/textures_dataset/test_A/Texture (1000).jpg'
'./datasets/textures_dataset/train_A/Texture (1001).jpg' -> './datasets/textures_dataset/test_A/Texture (1001).jpg'
'./datasets/textures_dataset/train_A/Texture (1002).jpg' -> './datasets/textures_dataset/test_A/Texture (1002).jpg'
'./datasets/textures_dataset/train_A/Texture (1003).jpg' -> './datasets/textures_dataset/test_A/Texture (1003).jpg'
'./datasets/textures_dataset/train_A/Texture (1004).jpg' -> './datasets/textures_dataset/test_A/Texture (1004).jpg'
'./datasets/textures_dataset/train_A/Texture (1005).jpg' -> './datasets/textures_dataset/test_A/Texture (1005).jpg'
'./datasets/textures_dataset/train_A/Texture (1006).jpg' -> './datasets/textures_dataset/test_A/Texture (1006).jpg'
'./datasets/textures_dataset/train_A/Texture (1007).jpg' -> './datasets/textures_dataset/test_A/Text

This is where it starts to get important to stay organized. This command creates a folder inside of "input_test_images" (which lives in your datasets folder). **Do this for each new experiment you create.**

In [ ]:
# create a folder + relevant subfolders for your new experiment
!mkdir ./datasets/input_test_images/$experiment_name
!mkdir ./datasets/input_test_images/$experiment_name/extracted_frames
!mkdir ./datasets/input_test_images/$experiment_name/canny_edges

### 6b. Running the generate command

Change your variables as need below, then run the cell to save the changes.

In [ ]:
##### REQUIRED: edit these each time each you generate images
loadSize = 1024 # set this to the width of your input images
fineSize = 1024 # set this to the height of your input images
how_many = 5  # The number of images you wish to generate. Make sure this is
                ## greater than or equal to the number of images you're trying to generate

##### OPTIONAL: change these if needed
which_epoch = 'latest' # The epoch you wish to generate images from (e.g. '20') (Defult: latest. I recommend this)


In [ ]:
# Run this cell to generate your images. This may take a few minutes.
!python test.py --name=$dataset_name --dataroot=./datasets/$dataset_name --checkpoints_dir checkpoints --results_dir=./results/$experiment_name --which_epoch=$which_epoch --how_many=$how_many --no_instance --label_nc 0 --loadSize=$loadSize --fineSize=$fineSize

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/stable_textures_dataset
display_winsize: 512
engine: None
export_onnx: None
feat_num: 3
fineSize: 1024
fp16: False
gpu_ids: [0]
how_many: 5
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_local_enhancers: 1
name: stable_textures_dataset
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: True
norm: instance
ntest: inf
onnx: None
output_nc: 3
phase: test
resize_or_crop: scale_width
results_dir: ./results/stablediffuse2normal_exp
serial_batches: False
tf_log: False
use_dropout: False
use_encoded_image: False
verbose: False
which_epoch: latest
-------------- End ------------